In [1]:
dir = 'YT_Data\YT'

In [2]:
!pip install xgboost

In [3]:
import numpy as np
import pandas as pd
import collections,os,json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.core.display import HTML,display
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=RuntimeWarning)
import pickle

C:\Users\mvemul2\AppData\Local\Temp\ipykernel_30668\4051890243.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML,display


In [4]:
def set_css():
      return display(HTML("""<style>
      #output-body {
        display: flex;
        align-items: center;
        justify-content: center;
        background-color:white;
        color:black;
        }
      .output_text pre{
        margin-right : 70px;
        color: black;
        font-weight: bold;
        /* font-size: 10px; */
      }

    }  </style>"""))

#Data Exploration

##Data Processing

In [5]:
def process_date_attributes(data):
    data["publish_time"] = pd.to_datetime(data["publish_time"])
    data["trending_date"] = pd.to_datetime(data["trending_date"],format="%y.%d.%m")
    data["trend_year"]=data["trending_date"].apply(lambda time:time.year)
    data["trend_date"] = data["trending_date"].apply(lambda time: time.day)
    data["trend_month"]=data["trending_date"].apply(lambda time:time.month)
    data["publish_year"]=data["publish_time"].apply(lambda time:time.year)
    data["publish_month"]=data["publish_time"].apply(lambda time:time.month)
    data["publish_day"]=data["publish_time"].apply(lambda time:time.day)
    data["publish_weekday"]=data["publish_time"].apply(lambda time:time.dayofweek)
    data["publish_hour"]=data["publish_time"].apply(lambda time:time.hour)
    return data

In [6]:
def match_categories(x,dictionary): #function to get the categories
    try:
        return dictionary[str(x)]
    except:
        return "Unknown"

##Data Cleaning

In [7]:
def data_cleaning(data):
    if data['description'].isnull().sum()>0:
        data['description'] = data['description'].replace(np.nan,"") #Replacing the NaN values in the "Description" column with an empty string
    data = data.sort_values('video_id').drop_duplicates(subset=['trending_date', 'title'], keep='last')
    data = data.drop(columns=["publish_time","category_id"],axis=1)
    return data

##Loading the datasets

In [8]:
def load_data(file_path): 
    ch_file=file_path
    Country_code = ch_file[0:2]
    path=os.path.join(dir,ch_file)
    input_df=pd.read_csv(path,encoding='utf-8')
    title_dict={}
    json_path=os.path.join(dir,(Country_code+str('_category_id.json')))
    json_input_df=pd.read_json(json_path)
    input_df['region']=Country_code
    for dict_a in json_input_df['items']:
        title_dict[dict_a['id']]=dict_a['snippet']['title'] 
    input_df = process_date_attributes(input_df) #Calling the function to process the date columns.
    input_df['category']=input_df['category_id'].apply(match_categories,args=(title_dict,))
    input_df = data_cleaning(input_df)
    return input_df

In [9]:
set_css()
con_list = []
con_dict = {}
t_list = []
t_df = pd.DataFrame()
for dirname, _, filenames in os.walk(dir):
    print("................................")
    for paths in filenames:
        if paths.lower().endswith(".csv"):
            con_list.append(paths[0:2])
            x = load_data(paths)
            t_list.append(x)
            x = x.drop(columns=['video_id','region'],axis=1)
            globals()[str(paths[0:2])] = x
            con_dict[str(paths[0:2])]=x
            print("------Created",paths[0:2],"DataFrame------")
    print("Processed all available Datasets")
t_df = pd.concat(t_list,ignore_index=True)
tdf = t_df
tdf.name = "The 6 Countries"
print("................................")
print("tdf  is the cumulative DataFrame")

................................
------Created CA DataFrame------
------Created DE DataFrame------
------Created FR DataFrame------
------Created GB DataFrame------
------Created IN DataFrame------
------Created US DataFrame------
Processed all available Datasets
................................
tdf  is the cumulative DataFrame


In [10]:
t_df.head()

,video_id,trending_date,title,channel_title,tags,views,likes,dislikes,comment_count,thumbnail_link,...,region,trend_year,trend_date,trend_month,publish_year,publish_month,publish_day,publish_weekday,publish_hour,category
0,--45ws7CEN0,2018-06-12,PlayStation E3 2018 Showcase | English,PlayStation Europe,"playstation|""playstation 4""|""playstation europ...",309197,3837,516,278,https://i.ytimg.com/vi/--45ws7CEN0/default.jpg,...,CA,2018,12,6,2018,6,12,1,3,Gaming
1,--7vNbh4UNA,2018-04-14,"Responding to ALL The Outrage, Ridiculous H3H3...",Philip DeFranco,"Elizabeth Hurley|""Instagram""|""Outrage""|""scanda...",1082647,52114,1284,10602,https://i.ytimg.com/vi/--7vNbh4UNA/default.jpg,...,CA,2018,14,4,2018,4,13,4,19,News & Politics
2,--7vNbh4UNA,2018-04-15,"Responding to ALL The Outrage, Ridiculous H3H3...",Philip DeFranco,"Elizabeth Hurley|""Instagram""|""Outrage""|""scanda...",1266423,58110,1504,11732,https://i.ytimg.com/vi/--7vNbh4UNA/default.jpg,...,CA,2018,15,4,2018,4,13,4,19,News & Politics
3,--7vNbh4UNA,2018-04-16,"Responding to ALL The Outrage, Ridiculous H3H3...",Philip DeFranco,"Elizabeth Hurley|""Instagram""|""Outrage""|""scanda...",1335225,60694,1576,10150,https://i.ytimg.com/vi/--7vNbh4UNA/default.jpg,...,CA,2018,16,4,2018,4,13,4,19,News & Politics
4,--MtKsH5oBY,2018-06-01,صحفي بين سبورت يكشف تفاصيل و كواليس استقالة زي...,RedsTech,"RedsTech|""bein sports""|""ريال مدريد""|""real madr...",511042,3517,372,767,https://i.ytimg.com/vi/--MtKsH5oBY/default.jpg,...,CA,2018,1,6,2018,5,31,3,12,Sports


In [11]:
def get_unique_categ(data):
    cat_x = data['category'].unique()
    cat_y = list(cat_x)
    if "Unknown" in cat_y:
        cat_y.remove('Unknown')
    dictc = {}
    for i in range(0,len(cat_y)):
        dictc[i] = cat_y[i]
    return dictc,cat_x
# dict_cat,cat_x = get_unique_categ(tdf)